In [2]:
import json

import requests


def get_parcelle_polygon(code_departement, code_commune, section, numero):
    """
    Récupère le polygone GeoJSON d'une parcelle cadastrale via l'API IGN.

    Args:
        code_departement (str): Ex: '75' pour Paris
        code_commune (str): Ex: '056' pour Paris 11e
        section (str): Ex: 'YF'
        numero (str): Ex: '0042'

    Returns:
        dict: GeoJSON de la parcelle, ou None si erreur.
    """
    url = f"https://apicarto.ign.fr/api/cadastre/parcelle/?code_insee={code_departement}{code_commune}&section={section}&numero{numero}"
    params = {
        "format": "geojson",  # Pour le polygone en GeoJSON
        "version": "flux",  # Données les plus récentes
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Lève une exception si erreur HTTP
        data = response.json()

        if data.get("type") == "FeatureCollection" and data.get("features"):
            return data
        else:
            print("Aucune parcelle trouvée pour cet ID.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erreur requête : {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Erreur JSON : {e}")
        return None


In [3]:
# Exemple d'utilisation : Parcelle fictive à Paris (adapte avec un vrai ID)
geojson = get_parcelle_polygon("24", "037", "DI", "0627")

if geojson:
    feature = geojson["features"][0]
    properties = feature["properties"]
    geometry = feature["geometry"]

    print("Parcelle trouvée !")
    print(f"ID complet : {properties.get('id', 'N/A')}")
    print(f"Commune : {properties.get('commune', 'N/A')}")
    print(
        f"Section/Numéro : {properties.get('section', 'N/A')}/{properties.get('numero', 'N/A')}"
    )
    print(f"Surface : {properties.get('surface', 'N/A')} m²")

    # Accès au polygone (coordonnées en WGS84/EPSG:4326)
    if geometry["type"] == "Polygon":
        coordinates = geometry["coordinates"][0]  # Premier anneau (externe)
        print(f"Nombre de points du polygone : {len(coordinates)}")
        print(f"Premier point (lon, lat) : {coordinates[0]}")

    # Optionnel : Sauvegarder en fichier GeoJSON
    with open("parcelle.geojson", "w", encoding="utf-8") as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)
    print("GeoJSON sauvegardé dans 'parcelle.geojson'.")
else:
    print("Impossible de récupérer la parcelle.")

Parcelle trouvée !
ID complet : N/A
Commune : N/A
Section/Numéro : DI/1029
Surface : N/A m²
GeoJSON sauvegardé dans 'parcelle.geojson'.
